# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [104]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.
'''

## 2. Generate the mathematical model

In [105]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [106]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location Problem

### Sets and Indices:

*   **I**: Set of customers, indexed by i
*   **J**: Set of potential facility locations, indexed by j

### Parameters:

*   **f_j**: Fixed annual cost of opening a facility at location j
*   **c_ij**: Transportation cost per unit of demand to service customer i from facility j
*   **d_i**: Annual demand of customer i
*   **u_j**: Maximum annual service volume capacity of facility at location j

### Decision Variables:

*   **x_ij**: Quantity of demand of customer i served by facility at location j 
*   **y_j**: Binary variable indicating whether a facility is opened at location j (1 if open, 0 otherwise)

### Objective Function:

Minimize the total cost, which is the sum of facility opening costs and transportation costs:

```
Minimize Z = ∑_(j∈J) f_j * y_j + ∑_(i∈I) ∑_(j∈J) c_ij * x_ij
```

### Constraints:

1.  **Demand Satisfaction**: Each customer's demand must be fully met:

    ```
    ∑_(j∈J) x_ij = d_i, ∀ i ∈ I
    ```
2.  **Capacity Limit**: The service volume at each facility cannot exceed its capacity:

    ```
    ∑_(i∈I) x_ij ≤ u_j * y_j, ∀ j ∈ J
    ```
3.  **Facility Usage**: Customers can only be served by open facilities:

    ```
    x_ij ≤ d_i * y_j, ∀ i ∈ I, ∀ j ∈ J
    ```
4.  **Binary Decision**: The decision to open a facility is binary:

    ```
    y_j ∈ {0, 1}, ∀ j ∈ J
    ```
5.  **Non-negativity**: The service volume cannot be negative:

    ```
    x_ij ≥ 0, ∀ i ∈ I, ∀ j ∈ J
    ```

This formulation represents a mixed-integer linear programming (MILP) problem, which can be solved using various optimization solvers to find the optimal facility locations and service allocations that minimize the total cost. 


## 3. Generate the pyomo code

In [107]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [108]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
I = range(1, 6)  # Set of customers
J = range(1, 4)  # Set of potential facility locations
f_j = {1: 10000, 2: 12000, 3: 8000}  # Fixed annual cost for each facility
c_ij = {
    (1, 1): 5, (1, 2): 6, (1, 3): 8,
    (2, 1): 4, (2, 2): 7, (2, 3): 5,
    (3, 1): 6, (3, 2): 3, (3, 3): 9,
    (4, 1): 7, (4, 2): 4, (4, 3): 3,
    (5, 1): 8, (5, 2): 5, (5, 3): 4,
}  # Transportation cost per unit of demand
d_i = {1: 200, 2: 250, 3: 180, 4: 220, 5: 150}  # Annual demand for each customer
u_j = {1: 500, 2: 400, 3: 600}  # Facility capacity

# Create a Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f_j = pyo.Param(model.J, initialize=f_j)
model.c_ij = pyo.Param(model.I, model.J, initialize=c_ij)
model.d_i = pyo.Param(model.I, initialize=d_i)
model.u_j = pyo.Param(model.J, initialize=u_j)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y_j = pyo.Var(model.J, domain=pyo.Binary)

# Objective Function
def total_cost(model):
    facility_cost = sum(model.f_j[j] * model.y_j[j] for j in model.J)
    transportation_cost = sum(model.c_ij[i, j] * model.x_ij[i, j] for i in model.I for j in model.J)
    return facility_cost + transportation_cost

model.objective = pyo.Objective(rule=total_cost, sense=pyo.minimize)

# Constraints
def demand_satisfaction(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == model.d_i[i]

model.demand_constraint = pyo.Constraint(model.I, rule=demand_satisfaction)

def capacity_limit(model, j):
    return sum(model.x_ij[i, j] for i in model.I) <= model.u_j[j] * model.y_j[j]

model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_limit)

def facility_usage(model, i, j):
    return model.x_ij[i, j] <= model.d_i[i] * model.y_j[j]

model.usage_constraint = pyo.Constraint(model.I, model.J, rule=facility_usage)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'cbc', 'gurobi', etc.
results = solver.solve(model)

# Print results (you might want to customize this based on your needs)
if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Optimal solution found!")
    print("Total cost:", pyo.value(model.objective))
    for j in model.J:
        if pyo.value(model.y_j[j]) == 1:
            print(f"Facility at location {j} is open.")
            for i in model.I:
                if pyo.value(model.x_ij[i, j]) > 0:
                    print(f"\tServing {pyo.value(model.x_ij[i, j])} units of demand from customer {i}")
else:
    print("Solver failed to find an optimal solution.")
```


## 4. Input problem data and try running the generated code

In [110]:
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
I = range(1, 6)  # Set of customers
J = range(1, 4)  # Set of potential facility locations
f_j = {1: 1000, 2: 1000, 3: 1000}  # Fixed annual cost for each facility
c_ij = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
} # Transportation cost per unit of demand
d_i = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180} # Annual demand for each customer
u_j = {1: 500, 2: 500, 3: 500}  # Facility capacity

# Create a Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f_j = pyo.Param(model.J, initialize=f_j)
model.c_ij = pyo.Param(model.I, model.J, initialize=c_ij)
model.d_i = pyo.Param(model.I, initialize=d_i)
model.u_j = pyo.Param(model.J, initialize=u_j)

# Decision Variables
model.x_ij = pyo.Var(model.I, model.J, domain=pyo.NonNegativeReals)
model.y_j = pyo.Var(model.J, domain=pyo.Binary)

# Objective Function
def total_cost(model):
    facility_cost = sum(model.f_j[j] * model.y_j[j] for j in model.J)
    transportation_cost = sum(model.c_ij[i, j] * model.x_ij[i, j] for i in model.I for j in model.J)
    return facility_cost + transportation_cost

model.objective = pyo.Objective(rule=total_cost, sense=pyo.minimize)

# Constraints
def demand_satisfaction(model, i):
    return sum(model.x_ij[i, j] for j in model.J) == model.d_i[i]

model.demand_constraint = pyo.Constraint(model.I, rule=demand_satisfaction)

def capacity_limit(model, j):
    return sum(model.x_ij[i, j] for i in model.I) <= model.u_j[j] * model.y_j[j]

model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_limit)

def facility_usage(model, i, j):
    return model.x_ij[i, j] <= model.d_i[i] * model.y_j[j]

model.usage_constraint = pyo.Constraint(model.I, model.J, rule=facility_usage)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose other solvers like 'cbc', 'gurobi', etc.
results = solver.solve(model)

# Print results (you might want to customize this based on your needs)
if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Optimal solution found!")
    print("Total cost:", pyo.value(model.objective))
    for j in model.J:
        if pyo.value(model.y_j[j]) == 1:
            print(f"Facility at location {j} is open.")
            for i in model.I:
                if pyo.value(model.x_ij[i, j]) > 0:
                    print(f"\tServing {pyo.value(model.x_ij[i, j])} units of demand from customer {i}")
else:
    print("Solver failed to find an optimal solution.")

Optimal solution found!
Total cost: 5610.0
Facility at location 2 is open.
	Serving 80.0 units of demand from customer 1
	Serving 270.0 units of demand from customer 2
	Serving 150.0 units of demand from customer 3
Facility at location 3 is open.
	Serving 7.78943796698235e-30 units of demand from customer 2
	Serving 100.0 units of demand from customer 3
	Serving 160.0 units of demand from customer 4
	Serving 180.0 units of demand from customer 5


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [111]:
print(response.text)

## Mathematical Optimization Model for Facility Location Problem

### Sets and Indices:

*   **I**: Set of customers, indexed by i
*   **J**: Set of potential facility locations, indexed by j

### Parameters:

*   **f_j**: Fixed annual cost of opening a facility at location j
*   **c_ij**: Transportation cost per unit of demand to service customer i from facility j
*   **d_i**: Annual demand of customer i
*   **u_j**: Maximum annual service volume capacity of facility at location j

### Decision Variables:

*   **x_ij**: Quantity of demand of customer i served by facility at location j 
*   **y_j**: Binary variable indicating whether a facility is opened at location j (1 if open, 0 otherwise)

### Objective Function:

Minimize the total cost, which is the sum of facility opening costs and transportation costs:

```
Minimize Z = ∑_(j∈J) f_j * y_j + ∑_(i∈I) ∑_(j∈J) c_ij * x_ij
```

### Constraints:

1.  **Demand Satisfaction**: Each customer's demand must be fully met:

    ```
    ∑_(j∈

In [112]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with your actual data)
I = range(1, 6)  # Set of customers
J = range(1, 4)  # Set of potential facility locations
f_j = {1: 10000, 2: 12000, 3: 8000}  # Fixed annual cost for each facility
c_ij = {
    (1, 1): 5, (1, 2): 6, (1, 3): 8,
    (2, 1): 4, (2, 2): 7, (2, 3): 5,
    (3, 1): 6, (3, 2): 3, (3, 3): 9,
    (4, 1): 7, (4, 2): 4, (4, 3): 3,
    (5, 1): 8, (5, 2): 5, (5, 3): 4,
}  # Transportation cost per unit of demand
d_i = {1: 200, 2: 250, 3: 180, 4: 220, 5: 150}  # Annual demand for each customer
u_j = {1: 500, 2: 400, 3: 600}  # Facility capacity

# Create a Pyomo model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.J = pyo.Set(initialize=J)

# Parameters
model.f_j = pyo.Param(model.J, initialize=f_j)
model.c_ij = pyo.Param(model.I, model.J, initialize=c_ij)
model.d_i = pyo.Param(model.I, initialize=d_i)
model.u_j = pyo.Param(model.J, initialize=u_j)

# Decision Variables
model.x_ij = pyo.V